<a href="https://colab.research.google.com/github/Chingkai-Lin/python-notes-machinelearning/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 自動分類新聞

In [ ]:
# # glob可以探測有哪些檔案
# import glob
# glob.glob("drive/My Drive/news/*")

['drive/My Drive/news/chinese_news_test.zip',
 'drive/My Drive/news/chinese_news_trans.zip']

In [ ]:
# 解壓縮檔案到目前的資料夾下
import zipfile
path = "drive/My Drive/news/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall()
path = "drive/My Drive/news/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall()

讀取，並切出「內容」以及「分類」

In [ ]:
import os
import glob
import pandas as pd

def read_data(base):
    contents, targets = [], []
    basedn = os.path.join(base, "*")
    for dn in glob.glob(basedn):
        upper = os.path.join(dn, "*.TXT")
        lower = os.path.join(dn, "*.txt")
        for fn in glob.glob(upper) + glob.glob(lower):
            with open(fn, "r", encoding="utf-8") as f:
                contents.append(f.read())
            target = os.path.split(dn)[-1]
            targets.append(target)
    df = pd.DataFrame({
        "content":contents,
        "target":targets
    })
    return df
train_df = read_data("chinese_news_trans")
test_df = read_data("chinese_news_test")

,content,target
0,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
1,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
2,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
3,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
4,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
...,...,...
96,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
97,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
98,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
99,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟


In [ ]:
import glob
glob.glob("chinese_news_trans/*")

['chinese_news_trans/環境',
 'chinese_news_trans/體育',
 'chinese_news_trans/交通',
 'chinese_news_trans/醫藥',
 'chinese_news_trans/軍事',
 'chinese_news_trans/藝術',
 'chinese_news_trans/計算機',
 'chinese_news_trans/教育',
 'chinese_news_trans/政治',
 'chinese_news_trans/經濟']

In [ ]:
# 處理資料 轉成0, 1, ...,9
# Series.replace({"環境:0}) 取出一列或一行，換成數字
# unique/value_counts
cat = train_df["target"].unique()
# enumerate搭配list
trans = {w:i for i, w in enumerate(cat)}
# 準備反向的字典
trans_r = {i:w for i, w in enumerate(cat)}
# 換成數字
y_train = train_df["target"].replace(trans)
y_test = test_df["target"].replace(trans)
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: target, Length: 101, dtype: int64

In [ ]:
import jieba
# 此處載入較大的辭典
from urllib.request import urlretrieve
if not os.path.exists("dict.txt.big"):
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
# type(train_df["content""])，可以使用type()確認是Series或是Dataframe
# 標點符號、\r、\n需要去除
p = train_df["content"][0]
" ".join(jieba.cut(p))

Building prefix dict from /content/dict.txt.big ...
Loading model from cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.299 seconds.
Prefix dict has been built successfully.


'                      《 瞭望 》 週刊 文章 ： 一個 全球性 的 環境 問題 — — 垃圾 \n 新華社 北京 ５ 月 １ ２ 日電 ５ 月 １ ４ 日 出版 的 第 ２ ０ 期 \n                           《 瞭望 》 週刊 將 發表 署名文章 《 一個 全球性 的 環境 問題 》 \n                           。 文章 摘要 如下 ： \n                                   目前 ， 世界 各國 拋棄 的 垃圾 越來越 多 ， 也 越來越 難處 \n                           理 。 據估計 ， 全球 每年 新增 的 垃圾 有 １ ０ ０ 億噸 左右 ， 人 \n                           均 ２ 噸 。 \n                                   世界 上 產生 垃圾 最多 的 是 美國 、 聯邦德國 、 英國 和 日 \n                           本 。 僅 美國 ， 每年 產生 工業 垃圾 ２ ０ 億噸 以上 ， 城市居民 \n                           垃圾 ２ · ２ 億噸 ， 丟棄 的 舊 汽車 １ ０ ０ ０ 多萬輛 ， 廢輪胎 \n                           上 億隻 ， 全國 垃圾 佔地面積 １ · １ ８ 萬平方公里 。 華盛頓 \n                           曾 有 一家 環保 機構 驚呼 ： 到 ９ ０ 年代 末 ， 美國 將有 一半 以 \n                           上 的 城市 無處 傾倒垃圾 ！ \n                                   有人 形象 地說 ， 垃圾 “ 無孔不入 ” 。 就連 被 認為 全球 \n                           淨土 的 珠穆朗瑪 峰 、 南極 ， 都 有 垃圾 “ 光顧 ” 。 近年來 ， \n                           垃圾 又 在 太空 “ 安家落戶 ” 。 自 １ ９ ５ ７ 年 蘇聯

In [ ]:
# 對全部做切詞
# apply(函式名字)
# 小括號: 函式沒有加上小括號的時候是給步驟，並未執行，會自行執行
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test

0      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
1      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
2      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
3      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
4      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
                             ...                        
96     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
97     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
98     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
99     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
100    日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
Name: content, Length: 101, dtype: object

In [ ]:
# 數次數
# fit 找出最小值和最大值
# fit_transform 轉換最小值0以及最大值1
# train: fit + transform
# test測試資料不做fit，只做transform，因為沒有出現過的資料有多少不重要！欄位不需重置
# 測試資料為根據train的fit資料做transform
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
# 下列的fit會把X_train的欄位存入到vec內
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
# vec.vocabulary_
# x_train_count
# print(x_train_count) #確認每欄有多少字來決定alpha要不要改
# vec.vocabulary_["\r\n"] # 確認標點符號是否在字典裡面

In [ ]:
# naive bayes 文字類型 multinomial
from sklearn.naive_bayes import MultinomialNB
# alpha若不調，預設值為1，依照資料量來調整alpha的數值
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [ ]:
# 混淆舉證 confusion_metrix
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index=[name + "(原本)" for name in target],
      columns=[name + "(預測)" for name in target],)

,環境(預測),體育(預測),交通(預測),醫藥(預測),軍事(預測),藝術(預測),計算機(預測),教育(預測),政治(預測),經濟(預測)
環境(原本),10,0,0,0,0,0,0,0,0,0
體育(原本),0,10,0,0,0,0,0,0,0,0
交通(原本),0,0,10,0,0,0,0,0,0,0
醫藥(原本),0,0,0,10,0,0,0,0,0,0
軍事(原本),0,0,0,0,10,0,0,0,0,0
藝術(原本),0,0,0,0,0,10,0,0,0,0
計算機(原本),0,0,0,0,0,0,10,0,0,0
教育(原本),0,0,0,0,0,0,0,10,0,0
政治(原本),0,0,0,0,0,0,0,0,11,0
經濟(原本),0,0,0,0,0,0,0,0,0,10


In [ ]:
# 輸入一則新聞
p = input("輸入一則新聞:")
# 轉換成次數[分詞]
x_predict = vec.transform([newscut(p)])
# predict_proba多少比率預測
proba = clf.predict_proba(x_predict)[0]
# zip 把list對應的
for w, p in zip(cat, proba):
    print(w, ":", round(p, 4))

輸入一則新聞:武漢肺炎》新增3例境外移入！2例同班機自美返台！ 新頭殼newtalk | 曾郡秋 綜合報導 發布 2020.10.06 | 14:52       媒體小農 A-A+ 中央流行疫情指揮中心發言人莊人祥   圖：中央流行疫情指揮中/提供 中央流行疫情指揮中心發言人莊人祥   圖：中央流行疫情指揮中/提供 今(6)日中央流行疫情指揮中心公布國內新增3例境外移入武漢肺炎(COVID-19)病例，其中2人為30多歲男性(案519、521)，1人為20多歲女性(案520)，均為本國籍，分別自美國及法國返台。其中案521與案519搭乘同班機自美返台，同班機前後二排座位旅客共11人居列為居家隔離對象。  發言人莊人祥表示，案519長期在美國工作，約2至3個月返台一次，在美國期間無不適症狀，10月4日返台，入境時主動告知搭機時出現喉嚨癢症狀，由機場檢疫人員協助採檢，於今日確診。  衛生單位已掌握個案同班機接觸者共21人，其中10人為前後二排座位旅客，列居家隔離對象，11人為機組員，因全程有適當防護，列自主健康管理對象。   莊人祥指出，案520今年已於法國留學畢業，9月中旬再前往法國處理私人事務，9月27日至30日陸續出現發燒、頭痛、流鼻水、鼻塞及嗅覺異常症狀，未於法國當地就醫；個案10月4日返國，入境時主動告知檢疫人員有流鼻水、鼻塞症狀，於機場進行採檢，並於今日確診。  目前衛生單位已掌握個案同班機接觸者共18人，其中7人為前後二排座位旅客，列居家隔離對象，11人為機組員，因全程有適當防護，列自主健康管理對象。  指揮中心表示，案521長期於美國工作(前次自台灣出境時間為去年5月)，今年10月3日起陸續出現腹瀉、喉嚨腫情形，10月4日返台，入境時主動告知檢疫人員有腹瀉情形，於機場進行採檢，並於今日確診。因本個案與案519搭乘同班機返台(兩人互不認識)，其中機組員7人已列為案519接觸者不重複匡列，而同班機前後二排座位旅客共11人則列為居家隔離對象。  指揮中心統計，國內截至目前累計95,089例武漢肺炎相關通報(含93,745例排除)，其中521例確診，分別為429例境外移入病例，55例本土病例，36例敦睦艦隊及1例不明。確診個案中7人死亡、485人解除隔離、29人住院隔離中。
環境 : 0.0
體育 : 0.0
交通 : 1.0
醫藥 : 0.0
軍事 : 0.